<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Load_pre_trained_weights_from_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow>=2.15.0 tqdm>=4.66

In [6]:
import tensorflow as tf
import numpy as np
import tqdm as tqdm
print(f"Tensorflow version:", tf.__version__)
print(f"tqdm version:",tqdm.__version__)

Tensorflow version: 2.19.0
tqdm version: 4.67.1


In [ ]:
from gpt_download3 import download_and_load_gpt2
settings , params = download_and_load_gpt2(model_size = "124M", model_dirs="gpt2")
print("Settings:",settings)
print("Parameter Disctionary Keys:",param.keys()) #blocks , b,g,wpe,wte
print(params["wte"])
print("Token embedding weight tensor dimesions:", param["wte"].shape)

In [ ]:
#define model configurations in a dictionary for compactness
model_confings ={
    "gpt-small (124M)":{"emb_dim":768,"n_layers":12,"n_heads":12},
    "gpt-medium (355M)":{"emb_dim":1024,"n_layers":24,"n_heads":16},
    "gpt-medium (774M)":{"emb_dim":1280,"n_layers":36,"n_heads":20},
    "gpt-medium (1558M)":{"emb_dim":1600,"n_layers":48,"n_heads":25},
}
#copy the base configuration and update with specific model settings
model_name = "gpt-small (124M)" #example model name
NEW_CONFIG = GPT_CONFIG_[model_name].copy()
NEW_CONFIG.update(model_confings[model_name])
print(NEW_CONFIG)

In [ ]:
NEW_CONFIG.update({"context_length":1024 , "qkv_bias": True})
gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [10]:
#check whether two tensor values left and right have the same dimenions . if shape is matching return the tensor
def assign(left , right):
  if left.shape != right.shape:
    raise ValueError(f"Shape mismatch: {left.shape} != {right.shape}")
  return torch.nn.Parameter(torch.tensor(right)) # Class for holding parameters in tensor that are trainable ....

In [11]:
import numpy as np
def load_weights_into_gpt(gpt,params):
  gpt.pos_emb.weight = assign(gpt.pos_emb.weight,params['wpe'])
  gpt.tok_emb.weight = assign(gpt.pos_emb.weight,params['wte'])

  for b in range(len(["blocks"])):
    q_w,q_k,q_v = np.split((params['blocks'][b]['attn']['c_attn'])["w"],3,axis=-1) #weights are fusion of q,k and v hence we need to split
    gpt.trf_blocks[b].attn.W_query.weight = assign(gpt.trf_blocks[b].attn.W_query.weight,q_w.T)
    gpt.trf_blocks[b].attn.W_key.weight = assign(gpt.trf_blocks[b].attn.W_key.weight,k_w.T)
    gpt.trf_blocks[b].attn.W_value.weight = assign(gpt.trf_blocks[b].attn.W_value.weight,v_w.T)

    #bias
    q_b,k_b,v_b = np.split((params['blocks'][b]['attn']['c_attn'])["b"],3,axis=-1)
    gpt.trf_blocks[b].attn.W_query.bias = assign(gpt.trf_blocks[b].attn.W_query.bias,q_b)
    gpt.trf_blocks[b].attn.W_key.bias = assign(gpt.trf_blocks[b].attn.W_key.bias,k_b)
    gpt.trf_blocks[b].attn.W_value.bias = assign(gpt.trf_blocks[b].attn.W_value.bias,v_b)

    #projection
    gpt.trf_blocks[b].attn.out_proj.weight = assign(gpt.trf_blocks[b].attn.out_proj.weight,params['blocks'][b]['attn']['c_proj']["w"].T)
    gpt.trf_blocks[b].attn.out_proj.bias = assign(gpt.trf_blocks[b].attn.out_proj.bias,params['blocks'][b]['attn']['c_proj']["b"])

    #layers
    gpt.trf_blocks[b].ff.layers[0].weight = assign(gpt.trf_blocks[b].ff.layers[0].weight,params['blocks'][b]['mlp']['c_fc']["w"].T)
    gpt.trf_blocks[b].ff.layers[0].bias = assign(gpt.trf_blocks[b].ff.layers[0].bias,params['blocks'][b]['mlp']['c_fc']["b"])
    gpt.trf_blocks[b].ff.layers[2].weight = assign(gpt.trf_blocks[b].ff.layers[2].weight,params['blocks'][b]['mlp']['c_proj']["w"].T)
    gpt.trf_blocks[b].ff.layers[2].bias = assign(gpt.trf_blocks[b].ff.layers[2].bias,params['blocks'][b]['mlp']['c_proj']["b"])

    #norm 1
    gpt.trf_blocks[b].norm1.scale = assign(gpt.trf_blocks[b].norm1.scale,params['blocks'][b]['ln_1']['g'])
    gpt.trf_blocks[b].norm1.bias = assign(gpt.trf_blocks[b].norm1.bias,params['blocks'][b]['ln_1']['b'])

    #norm 2
    gpt.trf_blocks[b].norm2.scale = assign(gpt.trf_blocks[b].norm2.scale,params['blocks'][b]['ln_2']['g'])
    gpt.trf_blocks[b].norm2.bias = assign(gpt.trf_blocks[b].norm2.bias,params['blocks'][b]['ln_2']['b'])

    #final scale and norm outside of transformer block
    gpt.final_norm.scale = assign(gpt.final_norm.scale,params['g'])
    gpt.final_norm.shift = assign(gpt.final_shift.shift,params['b'])
    #linear output later is also a NN
    gpt.out_head.weight = assign(gpt.out_head.weight,params['wte']) #weight tying


In [ ]:
load_weights_into_gpt(gpt,params) #load params into the GPT model that we created
gpt.to(device);

In [ ]:
torch.manual_seed(123)
token_ids = generate(
    model = gpt
    idx = text_to_token_ids("Every Effort Moves you",tokenizer).to(device)
    max_new_tokens = 25
    context_size = NEW_CONFIG["context_length"],
    top_k = 50, # 50 tokens have opportunity for maximum new generated tokens
    temperature = 1.5
)

print("Output text:\n",token_ids_to_text(token_ids,tokenizer))
